In [1]:
import os
import numpy as np
from numpy import array
import matplotlib
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [6]:
# from mockfactory import Catalog
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, ShapeFitPowerSpectrumTemplate, StandardPowerSpectrumTemplate
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, TNSTracerPowerSpectrumMultipoles, FOLPSTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.samplers.emcee import EmceeSampler
from desilike.samples import Profiles, plotting, Chain
from desilike import setup_logging
setup_logging()  # for logging messages

kmin     = 0.008
kmax     = 0.2
binning  = 0.006
k_ev     = np.arange(kmin, kmax+0.001, binning)
klim     = {ell*2: (kmin,kmax,binning) for ell in range(2)}

# the cosmology parameters
redshift    = 1.0
catalogue   = 'fiducial'  # fiducial, Mnu_p, Mnu_ppp -- QUIJOTE catalogue
r_pk        = 'RSD'  # RSD, LRG, QSQ, CATAS-- systematics
CovRsf      = 25  # -- covariance rescale factor
model       = 'FOLPS' # Kaiser, TNS, FOLPS
fitting     = 'FM' #FM, SF, STD
emulator_fn = f'./model/emulator_{model}_z{redshift}_{fitting}.npy'
chain_fn = f'./sampler_result/chain_{model}_V{CovRsf}_z{redshift}_{fitting}.npy'

if not os.path.exists(chain_fn):
    filename = []
    filedir = f'/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/{r_pk}_z{redshift}/npy/'
    for file in os.listdir(filedir):
        filename.append(filedir+file)
    covariance = filedir+'*'
    if model == 'Kaiser':
        theory = KaiserTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
    elif model == 'TNS':
        theory = TNSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
    elif model == 'FOLPS':
        theory = FOLPSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
    observable = TracerPowerSpectrumMultipolesObservable(data= filename,
                                                        covariance= covariance,
                                                        klim=klim,
                                                        theory=theory,
                                                        # kin=np.arange(0.001,0.35,0.002)
                                                        )
    likelihood = ObservablesGaussianLikelihood(observable, scale_covariance = 1/CovRsf) #
    likelihood()
    sampler = EmceeSampler(likelihood, seed=42, nwalkers=120, save_fn =chain_fn)
    sampler.run(check={'max_eigen_gr': 0.05}, max_iterations = 9001) # save every 300 iterations

In [3]:
# fitting compression 
import os
from desilike.observables.galaxy_clustering import ShapeFitCompressionObservable, StandardCompressionObservable
from desilike.emulators import Emulator, TaylorEmulatorEngine
from desilike.samplers.emcee import EmceeSampler
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.samples import Profiles, plotting, Chain
from desilike import setup_logging
setup_logging()  # for logging messages

fitting = 'STD'
compression_fn = f'./sampler_result/chain_FOLPS_V25_z1.0_{fitting}_compression'

if not os.path.exists(compression_fn):
    chain = Chain.load(f'./sampler_result/chain_FOLPS_V25_z1.0_{fitting}.npy').remove_burnin(0.5)
    print(chain)
    if fitting == 'SF':
        quantities = ['qap', 'qiso', 'df', 'dm']
        observable = ShapeFitCompressionObservable(data=chain, covariance=chain, z=1.0, quantities=quantities)
    elif fitting == 'STD':
        quantities = ['qap', 'qiso', 'df']
        observable = StandardCompressionObservable(data=chain, covariance=chain, z=1.0, quantities=quantities)
    emulator = Emulator(observable, engine=TaylorEmulatorEngine(order=3))
    emulator.set_samples()
    emulator.fit()
    likelihood = ObservablesGaussianLikelihood(observables=[emulator.to_calculator()])
    sampler = EmceeSampler(likelihood, seed=42, nwalkers=120, save_fn =compression_fn)
    sampler.run(check={'max_eigen_gr': 0.05}, max_iterations = 9001)

[000000.01] [0/1] 06-20 12:54  Chain                     INFO     Loading ./sampler_result/chain_FOLPS_V25_z1.0_STD.npy.
Chain(shape=(3000, 120), params=ParameterCollection(['qiso', 'qap', 'df', 'b1', 'b2', 'bs', 'alpha0', 'alpha2', 'sn0', 'sn2', 'logposterior', 'b3', 'alpha4', 'ct', 'loglikelihood', 'logprior']))
[000000.54] [0/1] 06-20 12:54  StandardCompressionObservable INFO     Found quantities ['qap', 'qiso', 'df'].
[000044.26] [0/1] 06-20 12:54  Emulator                  INFO     Varied parameters: ['h', 'omega_cdm', 'omega_b', 'logA'].
[000044.26] [0/1] 06-20 12:54  Emulator                  INFO     Found varying ['flattheory'] and fixed ['flatdata', 'covariance', 'quantities'] outputs.
[000050.25] [0/1] 06-20 12:54  Differentiation           INFO     Varied parameters: ['h', 'omega_cdm', 'omega_b', 'logA'].
[000081.30] [0/1] 06-20 12:55  Differentiation           INFO     Using finite-differentiation for parameter h.
[000087.03] [0/1] 06-20 12:55  Differentiation           IN

/Users/alain/opt/anaconda3/envs/desilike-env/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


[000531.44] [0/1] 06-20 13:02  Diagnostics               INFO     - effective sample size = (150 iterations / integrated autocorrelation time) is 12.
[000531.47] [0/1] 06-20 13:02  Diagnostics               INFO     - current mean acceptance rate is 0.595.
[000531.49] [0/1] 06-20 13:02  TaskManager               INFO     Entering TaskManager with 1 workers.
[000536.36] [0/1] 06-20 13:02  Chain                     INFO     Saving ./sampler_result/chain_FOLPS_V25_z1.0_STD_compression.
[000536.39] [0/1] 06-20 13:02  EmceeSampler              INFO     Diagnostics:
[000536.43] [0/1] 06-20 13:02  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 0.0189; < 0.05.
[000536.47] [0/1] 06-20 13:02  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.0054.
[000536.65] [0/1] 06-20 13:02  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 0.0231.
[000536.66] [0/1] 06-20 13:02  Diagnostics               INFO     - max Geweke is 0.154.
[00053

/Users/alain/opt/anaconda3/envs/desilike-env/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


[000537.07] [0/1] 06-20 13:02  Diagnostics               INFO     - effective sample size = (300 iterations / integrated autocorrelation time) is 14.9.
[000537.08] [0/1] 06-20 13:02  Diagnostics               INFO     - max variation of integrated autocorrelation time is 0.387.
[000537.08] [0/1] 06-20 13:02  Diagnostics               INFO     - current mean acceptance rate is 0.589.
